# Simulation of Mass Action Kinetics
This is an example notebook for interactive simulation of a user-defined chemical reaction network. 

In [ ]:
import sys
sys.path.insert(0, '../')
import numpy as np
from kinetics_simulator.guis import ProgressCurveGUI, Slider
from kinetics_simulator.chemicalkinetics import ChemicalReactionNetwork

# define a reaction dictionary for reactions modeled with mass action kinetics
# a reversible reaction may be defined with a "<->" character
# the forward rate constant of a given reaction must be provided first in the ordered dictionary
reaction_dict = {
    'S + E <-> SE': {'model': 'mass-action', 'rate-constants': [100 ,1], 'rate-constant-names': ['kon', 'koff']}, # binding of substrate to enzyme
    'SE -> P + E': {'model': 'mass-action', 'rate-constants': 100, 'rate-constant-names': 'kchem'} # chemical step
}

# define an initial state for the system
# if an initial value for a specie is not given, the code assigns it as zero
initial_values = {'S': 1000, 'E': 0.1}
reaction_network = ChemicalReactionNetwork(initial_values, reaction_dict, time=np.linspace(0,1000,10000))

# define a set of sliders for rate constants and specie concentrations
# see the definition of the Slider class in ./kinetics_simulator/guis.py
# for a description of args and kwargs
sliders = [
    Slider('kon', min=-6, max=6, stepsize=1), 
    Slider('koff', min=-6, max=6, stepsize=1),
    Slider('kchem', min=-6, max=6, stepsize=1),
    Slider('S', min=-6, max=6, stepsize=1),
    Slider('E', min=-6, max=6, stepsize=1),
    ]

# set multithread to True for parallel computing
figure = ProgressCurveGUI(reaction_network, sliders=sliders, multithread=True, title='Michaelis Menten Kinetic Mechanism')
figure.interactive()